# Tema 4

Link: https://profs.info.uaic.ro/~ancai/CN/lab/4/Tema%204.pdf

**Topics:**
* metoda **Jacobi** pentru rezolvarea de sisteme liniare $Ax=f$
  * unde $A$ este o matrice rara ($n$ suficient de mare) simetrica
  * si $det(A) \neq 0$ 
  
  
Resurse folosite pentru vizualizari: 
* Histogram: https://plotly.com/python/histograms/
* Slider: https://plotly.com/python/sliders/icelor

---


In fisierele (a_i.txt, b_i.txt, $i=1, \ldots, 5$ ) postate pe pagina laboratorului, sunt memorate pentru 5 sisteme liniare cu matrice rară (cu 'puține' elemente $\left.a_{i j} \neq \boldsymbol{0}\right)$ şi simetrică $\left(\boldsymbol{A}=\boldsymbol{A}^{T}\right), \boldsymbol{A x}=\boldsymbol{b}$, următoarele elemente:
- $\boldsymbol{n}$ dimensiunea sistemului,
- $\boldsymbol{a}_{i j} \neq \boldsymbol{0}, \boldsymbol{i}, \boldsymbol{j}(\boldsymbol{j} \leq \boldsymbol{i})$ - elementele nenule din partea inferior triunghiulară a matricei rare şi simetrice $A \in \mathbb{R}^{n \times n}$, indicii de linie și de coloană ai respectivului element,
- $\boldsymbol{b}_{i}, \boldsymbol{i}=1,2, \ldots, \boldsymbol{n}$ elementele vectorului termenilor liberi $b \in \mathbb{R}^{n}$.
1. Folosind fişierele atașate, să se citească dimensiunea sistemului, vectorul termenilor liberi şi să se genereze vectorii necesari pentru memorarea economică a matricei rare (se va folosi schema de memorare rară descrisă în Tema 3). Se presupune că elementele nenule ale matricei sunt plasate aleator în fişier (nu sunt ordonate după indicii de linie sau de coloană, sau altfel). Să se verifice că elementele de pe diagonala matricei sunt nenule. Se consideră dată precizia calculelor $\varepsilon=10^{-p}$.
2. Cu această memorare rară a matricei $\boldsymbol{A}$ să se aproximeze soluția sistemului liniar:
$$
A x=b
$$
folosind metoda Jacobi. Afişaţi numărul de iteraţii efectuate, în caz de convergenţă.
3. Să se verifice soluția calculată afişând norma:
$$
\left\|A x_{J}-b\right\|_{\infty}
$$
unde $\boldsymbol{x}_{J}$ este aproximarea soluției exacte obținută cu algoritmul Jacobi.

4. În toate calculele care includ matricea $\boldsymbol{A}$, se cere să se utilizeze memorarea rară a matricei (să nu se aloce în program nici o matrice clasică).
5. Calcularea vectorului de la pasul $\boldsymbol{k}, \boldsymbol{x}^{(\boldsymbol{k}+1)}$, să se facă cu o singură parcurgere a structurii rare asociate matricei $\boldsymbol{A}$. Ar fi util ca la citirea matricei din fişier şi construcţia structurii rare, să alocaţi un vector suplimentar $d$ care să memoreze elementele de pe diagonala matricei $\boldsymbol{A}$.

In [2]:
import numpy as np

# Rare Matrix implementation from tema3
from tema3 import RareMatrix

# Need to be implemented
from tema4 import ColumnVector, LinearSystem, plot_report

ModuleNotFoundError: No module named 'numpy'

### Metode iterative de rezolvare a sistemelor liniare

Pp. că $\boldsymbol{det(A)} \neq \mathbf{0}$, vom nota soluția exactă a sistemului cu $\boldsymbol{x}^{*}$ :
$$
\boldsymbol{x}^{*}:=\boldsymbol{A}^{-1} b .
$$
Metodele iterative de rezolvare a sistemelor liniare au fost deduse pentru sistemele de dimensiune 'mare' (n 'mare'), cu matricea sistemului $\boldsymbol{A}$, matrice rară (cu 'puţine' elemente $\boldsymbol{a}_{i j}$ nenule). În cazul metodelor iterative matricea $A$ nu se transformă (ca în cazul algoritmului de eliminare Gauss sau a descompunerilor $\boldsymbol{L} \boldsymbol{U}$ sau a factorizărilor $\boldsymbol{Q R}$ ) ci sunt folosite doar elementele nenule ale matricei pentru aproximarea soluției exacte $\boldsymbol{x}^{*}$. Pentru matricele rare se folosesc scheme de memorare economice specifice.

Pentru a aproxima soluția $\boldsymbol{x}^{*}$ se construiește un şir de vectori $\left\{\boldsymbol{x}^{(k)}\right\} \subset \mathbb{R}^{n}$ care, în anumite condiții, converge la soluția exactă $\boldsymbol{x}^{*}$ a sistemului (1):
$x^{(k)} \rightarrow x^{*}$, pentru $k \rightarrow \infty$

Vectorul $\boldsymbol{x}^{(0)}$ se inițializează, de obicei, cu 0 :
$$
x_{i}^{(0)}=0, i=1, \ldots, n
$$
Atunci când converge, limita șirului este chiar $\boldsymbol{x}^{*}$ soluția sistemului (1).

### Metoda Jacobi
Vom presupune că toate elementele diagonale ale matricei $\boldsymbol{A}$ sunt nenule:
$$
a_{i i} \neq 0, i=1, \ldots, n
$$
Când se citește matricea din fișier, se cere să se verifice dacă elementele diagonale ale matricei sunt nenule $\left(\left|a_{i i}\right|>\varepsilon, \forall i\right)$. Dacă există un element diagonal nul, nu se poate rezolva sistemul liniar folosind metoda iterativă Jacobi.
Şirul de vectori generat de metoda Jacobi este următorul:
$$
x_{i}^{(k+1)}=\frac{\left(b_{i}-\sum_{j=1}^{i-1} a_{i j} x_{j}^{(k)}-\sum_{j=i+1}^{n} a_{i j} x_{j}^{(k)}\right)}{a_{i i}}, i=1,2, \ldots, n
$$

#### Pseudocod
$$
\begin{aligned}
&x^{c}=x^{p}=0 ; \\
&k=0 ; \\
&\text { do } \\
&\left\{\quad x^{p}=x^{c} ;\right. \\
&\quad \text { calculează noul } x^{c} \text { folosind } x^{p} \text { (cu formula (3)); } \\
&\quad \text { calculează } \Delta x=\left\|x^{c}-x^{p}\right\| ; \\
&\quad k=k+1 ; \\
&\} \quad \text { while }\left(\Delta x \geq \varepsilon \text { şi } k \leq k_{\max } \text { şi } \Delta x \leq 10^{8}\right) / /\left(k_{\max }=10000\right) \\
&\text { if }(\Delta x<\varepsilon) x^{c} \approx x^{*} ; / / x^{c} \text { este aproximarea căutată a soluției } \\
&\text { else , divergență'; }
\end{aligned}
$$

Exemplu:

$
A=\left(\begin{array}{ccccc}
102.5 & 0.0 & 2.5 & 0.0 & 0.73 \\
0.0 & 104.88 & 1.05 & 0.0 & 0.33 \\
2.5 & 1.05 & 100.0 & 0.0 & 0.0 \\
0.0 & 0.0 & 0.0 & 101.3 & 1.5 \\
0.73 & 0.33 & 0.0 & 1.5 & 102.23
\end{array}\right), \quad b=\left(\begin{array}{c}
6.0 \\
7.0 \\
8.0 \\
9.0 \\
1.0
\end{array}\right)$

In [ ]:
a = RareMatrix(5, {
    0: {0: 102.5},
    1: {1: 104.88},
    2: {0: 2.5, 1: 1.05, 2: 100.0},
    3: {3: 101.3},
    4: {0: 0.73, 1: 0.33, 3: 1.5, 4: 102.23}
})
b = ColumnVector(5, [6.0,7.0,8.0,9.0, 1.0])
ls = LinearSystem(a, b)
x, deltas, k, xs = ls.solve_jacobi()
x

Validam utilizand norma $\left\|\boldsymbol{A} \boldsymbol{x}_{J}-\boldsymbol{f}\right\|_{\infty}$

In [ ]:
np.linalg.norm(ls.a * x - ls.b.values)

In [3]:
%%time
ls = LinearSystem.from_url(
        "http://profs.info.uaic.ro/~ancai/CN/lab/4/a_1.txt", 
        "http://profs.info.uaic.ro/~ancai/CN/lab/4/b_1.txt"
    )
print(f"Dimensiunea sistemului este {ls.a.n}")
x, deltas, k, xs = ls.solve_jacobi()
plot_report(x, deltas, k, xs)

NameError: name 'LinearSystem' is not defined

Validam utilizand norma $\left\|\boldsymbol{A} \boldsymbol{x}_{J}-\boldsymbol{f}\right\|_{\infty}$

In [4]:
np.linalg.norm(ls.a * x - ls.b.values)

NameError: name 'np' is not defined

In [ ]:
%%time
ls = LinearSystem.from_url(
        "http://profs.info.uaic.ro/~ancai/CN/lab/4/a_2.txt", 
        "http://profs.info.uaic.ro/~ancai/CN/lab/4/b_2.txt"
    )
print(f"Dimensiunea sistemului este {ls.a.n}")
x, deltas, k, xs = ls.solve_jacobi()
plot_report(x, deltas, k, xs)

Validam utilizand norma $\left\|\boldsymbol{A} \boldsymbol{x}_{J}-\boldsymbol{f}\right\|_{\infty}$

In [ ]:
np.linalg.norm(ls.a * x - ls.b.values)

In [ ]:
ls = LinearSystem.from_url(
        "http://profs.info.uaic.ro/~ancai/CN/lab/4/a_3.txt", 
        "http://profs.info.uaic.ro/~ancai/CN/lab/4/b_3.txt"
    )
print(f"Dimensiunea sistemului este {ls.a.n}")
x, deltas, k, xs = ls.solve_jacobi()
plot_report(x, deltas, k, xs)

Validam utilizand norma $\left\|\boldsymbol{A} \boldsymbol{x}_{J}-\boldsymbol{f}\right\|_{\infty}$

In [ ]:
np.linalg.norm(ls.a * x - ls.b.values)

In [ ]:
ls = LinearSystem.from_url(
        "http://profs.info.uaic.ro/~ancai/CN/lab/4/a_4.txt", 
        "http://profs.info.uaic.ro/~ancai/CN/lab/4/b_4.txt"
    )
print(f"Dimensiunea sistemului este {ls.a.n}")
x, deltas, k, xs = ls.solve_jacobi()
plot_report(x, deltas, k, xs)

Validam utilizand norma $\left\|\boldsymbol{A} \boldsymbol{x}_{J}-\boldsymbol{f}\right\|_{\infty}$

In [ ]:
np.linalg.norm(ls.a * x - ls.b.values)

In [ ]:
ls = LinearSystem.from_url(
        "http://profs.info.uaic.ro/~ancai/CN/lab/4/a_5.txt", 
        "http://profs.info.uaic.ro/~ancai/CN/lab/4/b_5.txt"
    )
print(f"Dimensiunea sistemului este {ls.a.n}")
x, deltas, k, xs = ls.solve_jacobi()
plot_report(x, deltas, k, xs)
deltas